In [ ]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21`

import $ivy.`org.apache.kafka::kafka:0.10.0.0`
import $ivy.`org.apache.spark::spark-core:2.0.0`
import $ivy.`org.apache.spark::spark-sql:2.0.0`
import $ivy.`org.apache.spark::spark-streaming:2.0.0`
import $ivy.`org.apache.spark::spark-streaming-kafka-0-10:2.0.0`
import $ivy.`com.datastax.spark::spark-cassandra-connector:2.0.1`

In [ ]:
import org.apache.kafka.clients.consumer.ConsumerRecord
import org.apache.kafka.common.serialization.StringDeserializer

import org.apache.spark.streaming.kafka010.{KafkaUtils, HasOffsetRanges, OffsetRange }
import org.apache.spark.streaming.kafka010.LocationStrategies.PreferConsistent
import org.apache.spark.streaming.kafka010.ConsumerStrategies.Subscribe
import org.apache.spark.streaming.{StreamingContext, Duration, Seconds}

import org.apache.spark.streaming.dstream.InputDStream

import org.apache.spark.{SparkContext, SparkConf, TaskContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types._

import com.datastax.spark.connector._

import java.net.InetAddress
import java.text.SimpleDateFormat
import java.util.Date

In [ ]:
val host_ip = InetAddress.getLocalHost().getHostAddress()

val conf = new SparkConf().setAppName("Streaming test").setMaster("local[1]").set("spark.cassandra.connection.host", host_ip)
val sc = new SparkContext(conf)
val sqlContext = new SQLContext(sc)

val ssc = new StreamingContext(sc, Seconds(5))

In [ ]:
val topics = Set("test")
val kafkaParams= scala.collection.immutable.Map[String, Object](
        "bootstrap.servers" -> "localhost:9092,127.0.0.1:2181",
        "key.deserializer" -> "org.apache.kafka.common.serialization.StringDeserializer",
        "value.deserializer" -> "org.apache.kafka.common.serialization.StringDeserializer",
        "group.id" -> "spark-streaming-consumer",
//         "auto.commit.interval.ms" -> "1000",
        "enable.auto.commit" -> (false: java.lang.Boolean),
        "auto.offset.reset" -> "latest")

In [ ]:
val stream: InputDStream[ConsumerRecord[String, String]] = KafkaUtils.createDirectStream[String, String](
      ssc,
      PreferConsistent,
      Subscribe[String, String](topics, kafkaParams)
    )

In [ ]:
val rows = stream.map(record => Row(record.value, new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date())))

In [ ]:

rows.foreachRDD { rdd =>
    if (!rdd.isEmpty()) {
        val struct = StructType(
            StructField("time_sent", StringType, true) ::
            StructField("time_received", StringType, true) :: Nil
        )
        
        sqlContext.createDataFrame(rdd, struct)
                  .write
                  .format("org.apache.spark.sql.cassandra")
                  .mode("append")
                  .options(scala.collection.Map("table" -> "sent_received", "keyspace" -> "test_time"))
                  .save()
        //rdd.saveToCassandra("test_time", "sent_received", SomeColumns("time_sent", "time_received"))
    }
}

rows.print()


In [ ]:
ssc.start()

In [ ]:
ssc.awaitTermination()